### 1 - Import libs

In [19]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
import geemap

# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h
import geopandas as gpd

### 2 - Basic Input Variables

In [20]:
# Area of Interest
country = 'Kenya'   # country case
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

# Grid Size (in degrees)
grid_size = 0.5

### 3 - Generate GRID and POINTS

In [21]:
# Generate the grid
grid = h.generate_grid(aoi, grid_size, grid_size, 0, 0).filterBounds(aoi)

# get a random point over each grid cell
randomInCell = grid.map(h.getRandomPoint)

# get the centroid of each gridcell
centerInCell = grid.map(h.getCenterPoint)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 5,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.setId)

### 4 - Display in map

In [22]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(aoi,6)

Map.addLayer(aoi)
Map.addLayer(grid)
Map.addLayer(centerInCell, {}, '1 Center Sample per gridcell (Systematic)')

#Map.addLayer(randomInCell, {}, '1 Random Sample per gridcell (Systematic random)')
#Map.addLayer(esaLc, null, 'ESA Worldcover')
#Map.addLayer(stratifiedSamples, {}, 'Stratified Samples per class from ESA LC')

Map

Map(center=[0.5336326464912589, 37.86026492779472], controls=(WidgetControl(options=['position'], widget=HBox(…

### 5 - Add auxillary data from global datasets

In [23]:
## Global Forest Change (Hansen et al., 2013)
gfc_col = ee.Image('UMD/hansen/global_forest_change_2020_v1_8').select(['treecover2000','loss','lossyear','gain'],['gfc_tc00','gfc_loss','gfc_year','gfc_gain'])

## ESA WorldCover 2020
esa_20  = ee.Image('ESA/WorldCover/v100/2020').rename('esa_lc20')

## Tropical Moist Forest - JRC 2021
tmf_annual= ee.ImageCollection('projects/JRC/TMF/v1_2020/AnnualChanges').mosaic()
tmf_annual_n = tmf_annual.rename(tmf_annual.bandNames().map(h.rename_TMF))

tmf_subtp = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_Subtypes').mosaic().rename('tmf_subtypes')
tmf_main  = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_MainClasses').mosaic().rename('tmf_main_cl')
tmf_deg   = ee.ImageCollection('projects/JRC/TMF/v1_2020/DegradationYear').mosaic().rename('tmf_deg_yr')
tmf_def   = ee.ImageCollection('projects/JRC/TMF/v1_2020/DeforestationYear').mosaic().rename('tmf_def_yr')

##  COMBINE COLLECTIONS
glo_ds = esa_20.addBands(gfc_col).addBands(tmf_subtp).addBands(tmf_main).addBands(tmf_deg).addBands(tmf_def).addBands(tmf_annual_n)

## EXTRACT THE DATA TO THE POINTS
columns = [
    'point_id','esa_lc20','gfc_tc00','gfc_loss','gfc_year','gfc_gain', 'tmf_main_cl','tmf_subtypes','tmf_1990','tmf_1995',
    'tmf_2000','tmf_2005','tmf_2010','tmf_2015','tmf_2020','tmf_def_yr','tmf_deg_yr',
]
centerPtsAux = glo_ds.reduceRegions(**{
  "reducer": ee.Reducer.first(),
  "collection": centerInCell
}).select([*columns, '.geo']);

### 6 EXPORT AS TABLE

In [25]:
## MAKE AS A GEODATAFRAME AND EXPORT
json = centerPtsAux.getInfo()
df = gpd.GeoDataFrame.from_features(json)
df = df.join(df["geometry"].apply(lambda p: list(p.coords)).explode().apply(pd.Series).rename(columns=({0:"LON", 1:"LAT"})))
df = df[['LAT', 'LON', *columns]]
df = df.sort_values("point_id", axis=0, ascending=False)
df.head()

LAT        LON point_id  esa_lc20  gfc_tc00  gfc_loss  gfc_year  \
26  0.525893  35.160179      100        40         0         0       NaN   
27  1.025892  35.160179      101        10        20         0       NaN   
28  1.525890  35.160179      102        20         1         0       NaN   
29  2.025888  35.160179      103        20         0         0       NaN   
30  2.525887  35.160179      104        20         4         0       NaN   

    gfc_gain  tmf_main_cl  tmf_subtypes  tmf_1990  tmf_1995  tmf_2000  \
26         0           70            91         6         6         6   
27         0           70            91         6         6         6   
28         0           70            91         6         6         6   
29         0           70            91         6         6         6   
30         0           70            91         6         6         6   

    tmf_2005  tmf_2010  tmf_2015  tmf_2020  tmf_def_yr  tmf_deg_yr  
26         6         6         6         6           0           0  
27         6         6         6         6           0           0  
28         6         6         6         6           0           0  
29         6         6         6         6           0           0  
30         6         6         6         6           0           0

In [26]:
## Export

folder = Path.home()/"downloads"/country.lower()/"data"
folder.mkdir(exist_ok=True, parents=True)
df.to_csv(folder/"points.csv", index=False)
